In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Read the Database
data = pd.read_csv('/Users/dltc2020/Documents/Senior-project/TheMotherLoad.csv')
# Turn Begin Date into DateTime
data['BeginDate'] = pd.to_datetime(data['BeginDate'])
#Create Lags for All of the different renewables
data['wind_output_lag1'] = data['Wind'].shift(1)
data['nuclear_output_lag1'] = data['Nuclear'].shift(1)
data['hydro_output_lag1'] = data['Hydro'].shift(1)

In [4]:
# Year Before
# Subtract one year using pd.DateOffset instead of pd.Timedelta
data['Previous_Year'] = data['BeginDate'] - pd.DateOffset(years=1)

# Extract only renewable data for Hydro, Nuclear, and Wind
Renewable_data = data[['BeginDate', 'Hydro', 'Nuclear', 'Wind']].copy()

# Define a function to get the closest renewable data from the previous year
def get_previous_day_renewables(row, reference_df):
    # Find the closest entry to the 'Previous_Year' in the reference DataFrame
    closest_entry = reference_df.iloc[(reference_df['BeginDate'] - row['Previous_Year']).abs().argsort()[:1]]
    
    if not closest_entry.empty:
        # Return the corresponding renewable values (Hydro, Nuclear, Wind)
        return closest_entry[['Hydro', 'Nuclear', 'Wind']].values[0]
    return [None, None, None]  # Return None for each renewable tech if no match is found

In [25]:
train_data = data[
    (data['BeginDate'] >= '2023-10-01') & 
    (data['BeginDate'] <= '2024-9-28')
]
# Define the start and end dates
start_date = '2024-09-28 00:00:00'
end_date = '2024-09-30 23:55:00'

# Generate a range of datetimes at 5-minute intervals
date_range = pd.date_range(start=start_date, end=end_date, freq='5T')

# Create a DataFrame with the date_range as a column
test_data = pd.DataFrame(date_range, columns=['Date Time'])


/var/folders/lp/l5v0z5894r912dc6qp90m2j00000gn/T/ipykernel_5045/3544552367.py:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq='5T')


In [26]:
#Hydro Test Set
hydro_test_data = test_data.copy()
hydro_test_data['Hydro'] = None
hydro_test_data['Hydro'] = hydro_test_data['Hydro'].fillna(train_data['Hydro'].mean())
#Hydro Nearest Neighbors
hydro_features = [ 'Hydro']
Hydro_X_train = train_data[hydro_features]
Hydro_Y_train = train_data['Hydro']
knn_regressor_hydro = KNeighborsRegressor(n_neighbors=6)
knn_regressor_hydro.fit(Hydro_X_train, Hydro_Y_train)
y_pred_hydro = knn_regressor_hydro.predict(hydro_test_data)

/var/folders/lp/l5v0z5894r912dc6qp90m2j00000gn/T/ipykernel_5045/2711509351.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hydro_test_data['Hydro'] = hydro_test_data['Hydro'].fillna(train_data['Hydro'].mean())


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Date Time


In [23]:
#Nuclear Test Set
nuke_test_data = test_data.copy()
nuke_test_data['Nuclear'] = None
nuke_test_data['Nuclear'] = nuke_test_data['Nuclear'].fillna(train_data['Nuclear'].mean())
#Nuclear Nearest Neighbor
nuke_features = [ 'Nuclear']
nuke_X_train = train_data[nuke_features]
nuke_Y_train = train_data['Nuclear']
knn_regressor_nuke = KNeighborsRegressor(n_neighbors=6)
knn_regressor_nuke.fit(nuke_X_train, nuke_Y_train)
y_pred_nuke = knn_regressor_nuke.predict(nuke_X_train)

/var/folders/lp/l5v0z5894r912dc6qp90m2j00000gn/T/ipykernel_5045/159151801.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  nuke_test_data['Nuclear'] = nuke_test_data['Nuclear'].fillna(train_data['Nuclear'].mean())


In [ ]:
wind_test_data = test_data.copy()
wind_test_data['Wind'] = None
wind_test_data['Wind'] = hydro_test_data['Wind'].fillna(train_data['Wind'].mean())

In [ ]:
wind_features = ['wind_output_lag1', 'windspeed', 'windgust', 'cloudcover', 'severerisk', 'precip', 'humidity', 'Previous_Day_Wind']
wind_X_train = train_data[wind_features]
wind_Y_train = train_data['Wind']
regression = LinearRegression()
regression.fit(wind_X_train, wind_Y_train)
predictions = regression.predict(X_test)